In [1]:
import re
import os
import xlrd
import pickle
import pandas as pd
import numpy as np
from unicodedata import normalize, combining

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='luizhemelo', timeout=10)

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
# city location cache
path = "../data/cities_cache.pickle"
if os.path.exists(path):
    file = open(path, 'rb')
    cities = pickle.load(file)
    file.close()
else:
    cities = {
        'poxereu':[-15.8290891, -54.3925375, "mato grosso"],
        'brasilia':[-10.3333333, -53.2, "distrito federal"]
    }
cols = ['Municipio']

def convert_temp(temp):
    if type(temp) == str:
        temp = temp.replace(",","")
        temp = temp.replace(".","")
        temp = re.sub('\(([^\)]+)\)', '', temp)
        return temp
    return temp

def format_name(name: str):
    """
    Format name to remove punctuation and spaces
    """
    name = re.sub("([^a-zà-üA-ZÀ-Ü0-9])", " ", name)
    nfkd_str = normalize("NFKD", name)
    name = "".join(
        [c for _, c in enumerate(nfkd_str) if not combining(c)]
    ).lower()
    while "  " in name:
        name = name.replace("   ", " ")
        name = name.replace("  ", " ")
    return name

def preprocess_dataset(df: pd.DataFrame):
    ctt = df[cols].value_counts().index.tolist()
    df = pd.DataFrame({'Municipio':ctt})
    
    latitude = []
    longitude = []
    
    for i,row in df.iterrows():
        city = row['Municipio'][0]
        df.loc[i,'Municipio'] = city
        
        if city not in list(cities.keys()):
            location = geolocator.geocode(str(city+',Brazil'))
            
            cities[city] = [location.raw['lat'], location.raw['lon']]
            
            latitude.append(location.raw['lat'])
            longitude.append(location.raw['lon'])
        else:
            latitude.append(cities[city][0])
            longitude.append(cities[city][1])
    
    df['Latitude'] = latitude
    df['Longitude'] = longitude
    
    return df

## Carregamento e tratamentos básicos dos datasets

In [3]:
# Load and treat meteorological dataset
df_meteorological = pd.read_csv(f"../data/data_meteorological.csv", index_col=0)
df_meteorological

,Municipio,Temperatura_media,Latitude,Longitude
0,guarda mor,23.356669,-17.561389,-47.199167
1,corumba,26.282203,-18.996667,-57.637500
2,vila velha,23.504628,-20.466944,-40.403889
3,tupa,23.506298,-21.927251,-50.490251
4,rio brilhante,23.428128,-21.774944,-54.528108
...,...,...,...,...
583,campos lindos,28.661224,-8.154722,-46.639444
584,barra,27.322538,-11.084722,-43.138889
585,salvador,25.592511,-13.005515,-38.505760
586,indaial,21.290087,-26.913611,-49.268056


In [4]:
# Load and treat Covid-19 dataset
df_covid = pd.read_csv(f"../data/data_covid.csv", low_memory=False)[["Município", "CasosNovos"]]
df_covid["Município"] = df_covid["Município"].astype("str")
df_covid["CasosNovos"] = df_covid["CasosNovos"].astype("int")
df_covid["Município"] = df_covid["Município"].apply(format_name)
df_covid = df_covid.groupby(["Município"], as_index=False)["CasosNovos"].sum()
df_covid

,Município,CasosNovos
0,abadia de goias,1238
1,abadia dos dourados,491
2,abadiania,412
3,abaete,1411
4,abaetetuba,4805
...,...,...
5285,xique xique,2053
5286,zabele,95
5287,zacarias,133
5288,ze doca,1523


In [5]:
# Load and treat population dataset
df_population = pd.read_csv(f"../data/data_population.csv", sep=";")[["NOME DO MUNICÍPIO", "POPULAÇÃO ESTIMADA"]]
df_population["NOME DO MUNICÍPIO"] = df_population["NOME DO MUNICÍPIO"].apply(format_name)
df_population = df_population.groupby(["NOME DO MUNICÍPIO"], as_index=False)["POPULAÇÃO ESTIMADA"].max()
df_population

,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
0,abadia de goias,"9,158"
1,abadia dos dourados,"7,022"
2,abadiania,"20,873"
3,abaete,"23,263"
4,abaetetuba,"160,439"
...,...,...
5284,xique xique,"46,562"
5285,zabele,"2,269"
5286,zacarias,"2,784"
5287,ze doca,"52,190"


In [6]:
# Load and treat IBGE code dataset
wb = xlrd.open_workbook(f"../data/data_codigos_ibge.xls", logfile=open(os.devnull, 'w'), encoding_override='ISO-8859-1')
df_code = pd.read_excel(wb, header=6)[["Nome_Município","Código Município Completo"]]
df_code["Nome_Município"] = df_code["Nome_Município"].apply(format_name)
df_code

,Nome_Município,Código Município Completo
0,alta floresta d oeste,1100015
1,alto alegre dos parecis,1100379
2,alto paraiso,1100403
3,alvorada d oeste,1100346
4,ariquemes,1100023
...,...,...
5565,vianopolis,5222005
5566,vicentinopolis,5222054
5567,vila boa,5222203
5568,vila propicio,5222302


## Criação do Banco de Dados Geográficos

In [7]:
df_pop_covid = df_covid.merge(
    right=df_population,
    left_on='Município',
    right_on='NOME DO MUNICÍPIO',
    how="inner",
    sort=True,
    copy=True
).drop(["NOME DO MUNICÍPIO"], axis=1)
df_pop_covid.columns = ["Municipio", "Novos_casos", "Populacao_estimada"]
df_pop_covid

,Municipio,Novos_casos,Populacao_estimada
0,abadia de goias,1238,"9,158"
1,abadia dos dourados,491,"7,022"
2,abadiania,412,"20,873"
3,abaete,1411,"23,263"
4,abaetetuba,4805,"160,439"
...,...,...,...
5283,xique xique,2053,"46,562"
5284,zabele,95,"2,269"
5285,zacarias,133,"2,784"
5286,ze doca,1523,"52,190"


In [8]:
df_pop_covid = df_pop_covid.merge(
    right=df_code,
    left_on='Municipio',
    right_on='Nome_Município',
    how="inner",
    sort=True,
    copy=True
).drop(["Nome_Município"], axis=1)
df_pop_covid.columns = ["Municipio", "Novos_casos", "Populacao_estimada", "id"]
df_pop_covid["id"] = df_pop_covid["id"].astype("str")
df_pop_covid

,Municipio,Novos_casos,Populacao_estimada,id
0,abadia de goias,1238,"9,158",5200050
1,abadia dos dourados,491,"7,022",3100104
2,abadiania,412,"20,873",5200100
3,abaete,1411,"23,263",3100203
4,abaetetuba,4805,"160,439",1500107
...,...,...,...,...
5563,xique xique,2053,"46,562",2933604
5564,zabele,95,"2,269",2517407
5565,zacarias,133,"2,784",3557154
5566,ze doca,1523,"52,190",2114007


In [9]:
lat_long_df = preprocess_dataset(df_pop_covid)
lat_long_df

,Municipio,Latitude,Longitude
0,sao domingos,-11.4636112,-39.526527
1,bom jesus,-28.6692723,-50.4346894
2,vera cruz,-22.2212638,-49.8243021
3,santa helena,-24.8591735,-54.3328813
4,santa terezinha,-26.7813,-50.009017
...,...,...,...
5282,garuva,-26.0294904,-48.8545202
5283,garruchos,-28.1892458,-55.6382958
5284,garrafao do norte,-1.9282256,-47.0513157
5285,garopaba,-28.0274798,-48.6240197


In [10]:
df2_pop_covid = df_pop_covid.merge(
    right=lat_long_df,
    left_on='Municipio',
    right_on='Municipio',
    how="inner",
    sort=True,
    copy=True
).reset_index(drop=True)
df2_pop_covid["Novos_casos"] = df2_pop_covid["Novos_casos"].astype("int")
df2_pop_covid["Populacao_estimada"] = df2_pop_covid["Populacao_estimada"].apply(convert_temp).astype("int")
df2_pop_covid["Latitude"] = df2_pop_covid["Latitude"].astype("float")
df2_pop_covid["Longitude"] = df2_pop_covid["Longitude"].astype("float")

In [11]:
def treat_negatives(value):
    if value < 0:
        return 0
    return value

df2_pop_covid["Novos_casos"] = df2_pop_covid["Novos_casos"].apply(treat_negatives)
df2_pop_covid["Relacao_perc_pop_covid"] = np.round((df2_pop_covid["Novos_casos"].values * 100) / df2_pop_covid["Populacao_estimada"].values, 4)

df2_pop_covid

,Municipio,Novos_casos,Populacao_estimada,id,Latitude,Longitude,Relacao_perc_pop_covid
0,abadia de goias,1238,9158,5200050,-16.758400,-49.438300,13.5182
1,abadia dos dourados,491,7022,3100104,-18.491119,-47.406359,6.9923
2,abadiania,412,20873,5200100,-16.194095,-48.706777,1.9738
3,abaete,1411,23263,3100203,-19.156683,-45.448121,6.0654
4,abaetetuba,4805,160439,1500107,-1.721828,-48.878843,2.9949
...,...,...,...,...,...,...,...
5563,xique xique,2053,46562,2933604,-10.821719,-42.726569,4.4092
5564,zabele,95,2269,2517407,-12.355532,-41.664478,4.1869
5565,zacarias,133,2784,3557154,-21.052365,-50.051450,4.7773
5566,ze doca,1523,52190,2114007,-3.270145,-45.655344,2.9182


In [12]:
if os.path.exists(path) == False:
    file = open(path, 'wb')
    pickle.dump(cities, file)
    file.close()

df2_pop_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Municipio               5568 non-null   object 
 1   Novos_casos             5568 non-null   int64  
 2   Populacao_estimada      5568 non-null   int64  
 3   id                      5568 non-null   object 
 4   Latitude                5568 non-null   float64
 5   Longitude               5568 non-null   float64
 6   Relacao_perc_pop_covid  5568 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 304.6+ KB


In [13]:
df2_pop_covid.to_csv(f"../data/preprocessed_pop_covid.csv")